# Imports

In [2]:
import cv2
import matplotlib.pyplot as plt
import copy
import numpy as np
import json
import glob
import pandas as pd

import model
import util
from body import Body

d:\data_science\python\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
d:\data_science\python\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
d:\data_science\python\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


# Matching keypoints from model to UAV Human Dataset

In [3]:
# Index generated OpenPose model: Body Part name in UAV dataset

body_part_index = {0: 'nose',
#1: The model generates a neck but UAV doesn't have coordinates for neck
2: 'rightShoulder',
3: 'rightElbow',
4: 'rightWrist',
5: 'leftShoulder', 
6: 'leftElbow',
7: 'leftWrist',
8: 'rightHip',
9: 'rightKnee',
10: 'rightAnkle',
11: 'leftHip',
12: 'leftKnee',
13: 'leftAnkle',
14: 'rightEye',
15: 'leftEye',
16: 'rightEar',
17: 'leftEar'}

# Reverse the list to inverse reference
index_body_part = {v: k for k, v in body_part_index.items()}

# Defining Metric

In [4]:
def mAP(true_points, pred_points, thresh=5):
    
    # Initialize the precision and recall arrays
    precision = np.zeros(len(true_points))
    recall = np.zeros(len(true_points))
    
    pred_coords = []
    true_coords = []
    
    for key in true_points.keys():
        true_coords.append(true_points[key])
        pred_coords.append(pred_points[key])
    
    # Iterate over each body part
    for i in range(len(true_points)):
        # Calculate the Euclidean distance between the predicted and ground truth coordinates for the body part
        dist = np.linalg.norm(np.array(pred_coords[i]) - np.array(true_coords[i]))

        # Assign a label of 1 (true positive) or 0 (false positive) based on whether the distance is below or above the threshold
        label = int(dist <= thresh)

        # Update the precision and recall arrays
        if label == 1:
            precision[i] = 1
            recall[i] = 1
        else:
            precision[i] = 0
            recall[i] = 0

    # Calculate the cumulative precision and recall arrays
    cum_precision = np.cumsum(precision) / (np.arange(len(true_points)) + 1)
    cum_recall = np.cumsum(recall) / len(true_points)

    # Calculate the average precision
    ap = np.sum((cum_recall[1:] - cum_recall[:-1]) * cum_precision[1:])

    return ap

# Essential Functions

In [5]:
# Load pre-trained models
body_estimation = Body('body_pose_model.pth')

# Denormalizing function to match UAV Human Dataset coordinate format
def denorm(x, y, img):
    w, h, _ = img.shape
    d_x = (x/h)*100
    d_y = (y/w)*100
    
    return d_x, d_y

# Function to read JSON file
def read_json(json_file):
    # Dictionaries to hold body parts and its corresponding coordinates
    part_dict = {}
    corr_part_dict = {}
    
    # Opening JSON file
    f = open(json_file)

    # returns JSON object as 
    # a dictionary
    data = json.load(f)

    # Closing file
    f.close()
    
    # Extracts body parts from UAV Human file specific JSON file
    for part in data['completions'][0]['result']:
        if part['value']['keypointlabels'][0] in body_part_index.values():
            part_dict[part['value']['keypointlabels'][0]] = [part['value']['x'], part['value']['y']]
    
    # Checks if every body part is present in the file, else fills it with [0.0, 0.0]
    for body_part in body_part_index.values():
        if body_part in part_dict.keys():
            corr_part_dict[body_part] = part_dict[body_part]
        else:
            corr_part_dict[body_part] = [0.0, 0.0]

    return corr_part_dict
    

# Function to predict keypoints and draw it on an image
def predict_n_draw(image_path, labels, zero_ind):
    # Reading an image
    oriImg = cv2.imread(image_path)  # B,G,R order
    coords = {}
    
    # Generate Candidate and Subset
    # Candidate contains coordinates of the body parts along with the index and score
    # Subset contains information about different people in the image and which point belongs to which subject
    # along with the count and score
    candidate, subset = body_estimation(oriImg)
    
    # If in case, the model was not able to predict anything, and the candidate list is empty
    if np.shape(candidate)[0] == 0:
        canvas = copy.deepcopy(oriImg)
        plt.imshow(canvas[:, :, [2, 1, 0]])
        plt.axis('off')
        plt.show()
        return None, None

    # Denormalizing points to match the format of UAV Human Dataset
    x, y = denorm(candidate[:, 0], candidate[:, 1], oriImg)
    
    # If the true label of a body part is [0,0], i.e. it doesn't exist in the data
    for i in range(len(x)):
        if i in zero_ind:
            coords[body_part_index[i]] = [0.0, 0.0]
        elif i in labels:
            coords[body_part_index[i]] = [x[i], y[i]]
    
    # Draw the image and the points on it
    canvas = copy.deepcopy(oriImg)
    canvas = util.draw_bodypose(canvas, candidate, subset)
    
    plt.imshow(canvas[:, :, [2, 1, 0]])
    plt.axis('off')
    plt.show()
    return coords, subset


# Read test files

In [6]:
# Read file names from the test file
with open(r'..\\test_files.txt', 'r') as f:
    test_files = f.readlines()
f.close()

test_files = [i[:-1] for i in test_files]

# Generate Body Keypoints

In [ ]:
# List to store image names
img_names = []

# List to store the class of the files
class_names = []

# List to store the mAP values
mAP_list = []

# Individual list to store the predicted values of X and Y 
# coordinates of the body parts
pred_x = []
pred_y = []

# Individual list to store the ground truth values of X and Y 
# coordinates of the body parts
true_x = []
true_y = []

# Class path
folder_path = "..\\classes\\"

# Count of number of files
cnt = 0
        
# Iterate over the files and generate keypoints
for filepath in test_files:
    # Extract class and image name
    class_name = filepath.rsplit("\\")[-2]
    img_name = filepath.rsplit("\\")[-1]
    print("Class name:", class_name)
    print("Image Name:", img_name)
    
    # List to capture parts that don't exist in groudn truth
    zero_ind = []

    # Get JSON and image path
    label_path = folder_path+filepath.rsplit("\\")[-2]+"\\"+filepath.rsplit('\\')[-1].rsplit('.')[0]+".json"
    img_path = folder_path+filepath.rsplit("\\")[-2]+"\\"+img_name

    try:
        # Get keypoints coordinates for each body part
        true_labels = read_json(label_path)
        
        # If the part doesn't exist in groudn truth, get its index
        for key in true_labels.keys():
            if true_labels[key] == [0.0, 0.0]:
                zero_ind.append(index_body_part[key])

        # List to identify the body parts labeled within the image
        true_label_present_list = [index_body_part[i] for i in list(true_labels.keys())]
        
        # Run OpenPose and get keypoints
        pred_labels, subset = predict_n_draw(img_path, true_label_present_list, zero_ind)

        # If model output is empty
        if pred_labels is None and subset is None:
            print("mAP: Undefined", cnt)
            cnt += 1
            img_names.append(img_name)
            class_names.append(class_name)
            mAP_list.append(0.0)
        else:
            # If the body part in ground truth has coordinate [0.0, 0.0], then make
            # predicted label 0 as well
            for key in true_labels.keys():
                if key not in pred_labels.keys():
                    pred_labels[key] = [0.0, 0.0]

            # Calculate mAP
            output = mAP(true_labels, pred_labels)
            # Append it to mAP list to keep count
            mAP_list.append(output)
            print("mAP:", output, cnt)
        
            cnt += 1
            img_names.append(img_name)
            class_names.append(class_name)
            # Get coordinates for MLP
            # Predicted coordinates
            pred_x.append([i[0] for i in list(pred_labels.values())])
            pred_y.append([i[1] for i in list(pred_labels.values())])
            # True coordinates
            true_x.append([i[0] for i in list(true_labels.values())])
            true_y.append([i[1] for i in list(true_labels.values())])
            print("Number of true labels:", len(true_labels))
            print("Number of pred labels:", len(pred_labels))
        print("-"*30)
    except Exception as e:
        # As a safety measure 
        print("Error:", e)
        img_names.append(img_name)
        class_names.append(class_name)
        mAP_list.append(-1)
        print("-"*30)

# Store results

In [8]:
# Store class name, image name and corresponding mAP values and store it in a datset
results_df = pd.DataFrame({'class_name': class_names, 'image_name':img_names, 'mAP': mAP_list})
results_df.to_csv("openpose_baseline_test_results.csv", index=False)

In [9]:
print("Mean mAP value of the entire dataset:", np.mean(results_df['mAP']))

Mean mAP value of the entire dataset: 0.6334673667218156


In [10]:
# Store the true and predicted coordinates of body keypoints
coords_df = pd.DataFrame({'true_x': true_x, 'true_y':true_y, 
                          'pred_x': pred_x, 'pred_y':pred_y})
coords_df.to_csv("test_coords.csv", index=False)